In [3]:
import shutil
import sys
import os
from pathlib import Path
import json
import re

1. Add Path and read movie list

In [4]:
root_path = Path('/home/jianghui/dataset/MovieNet-Subset/')

tt_list = []
with open(Path(root_path, 'movie_id_list'), 'r') as f_tt_list:
    for tt_line in f_tt_list:
        tt_list.append(tt_line.replace('\n', ''))

In [ ]:
ori_path = Path('/home/jianghui/dataset/MovieNet/')

2. Read shot.json file

In [ ]:
DEBUG = False

for tt_id in (tt_list if not DEBUG else ['tt0047396']):
    print(f'processing {tt_id} ...')
    # read shot file
    shot_path = Path(root_path, 'all/shot/', f'{tt_id}.json')
    with open(shot_path, 'r') as f_shot:
        shot_json = json.load(f_shot)
    
    # make up subtitle file
    subtitle_json = []
    for shot_element in shot_json:
        subtitle_idx = 0
        if shot_element['subtitle_start_id'] == None : continue 
        for i, subtitle_id in enumerate(range(int(shot_element['subtitle_start_id']), int(shot_element['subtitle_end_id'] + 1))):
            if subtitle_id < subtitle_idx: continue
            subtitle_element = {'subtitle_id':subtitle_id, 'scentence':shot_element['subtitle'][i]}
            subtitle_json.append(subtitle_element)
            subtitle_idx += 1

    # save subtitle file
    subtitle_path = Path(root_path, 'all/subtitle', f'{tt_id}.json')
    with open(subtitle_path, 'w') as f_subtitle:
        json.dump(subtitle_json, f_subtitle, indent=2)

3. clean shot.json file

In [ ]:
DEBUG = False

for tt_id in (tt_list if not DEBUG else ['tt0047396']):
    print(f'processing {tt_id} ...')
    # read shot file
    shot_path = Path(root_path, 'all/shot/', f'{tt_id}.json')
    with open(shot_path, 'r') as f_shot:
        shot_json = json.load(f_shot)
    
    for shot_element in shot_json:
        if shot_element['subtitle_id'][0] == None:
            shot_element['subtitle_id'] = []
        # if "subtitle" not in shot_element: continue

        # # change subtitle_id format
        # subtitle_id = []
        # subtitle_id.append(shot_element['subtitle_start_id'])
        # subtitle_id.append(shot_element['subtitle_end_id'])
        # shot_element['subtitle_id'] = subtitle_id

        # # pop subtitle
        # shot_element.pop('subtitle')
        # shot_element.pop('subtitle_start_id')
        # shot_element.pop('subtitle_end_id')
    
    if DEBUG : print(shot_json)
    # save shot file
    with open(shot_path, 'w') as f_shot:
        json.dump(shot_json, f_shot, indent=2)

    

4. re_order tt_list

In [ ]:
tt_list.sort()
tt_list

In [ ]:
with open(Path(root_path, 'movie_id_list'), 'w') as f_tt_list:
    for tt_line in tt_list:
        f_tt_list.write(tt_line + '\n')

5. add movie_info with subtitle_num, shot_num, scene_num, synopsis_num 

In [ ]:
info_path = Path(root_path, 'movie_info.json')
with open(info_path, 'r') as f_info:
    info_json = json.load(f_info)

In [ ]:
for info_elem_key in info_json:
    info_json[info_elem_key]['subtitle_num'] = None
    info_json[info_elem_key]['shot_num'] = None
    info_json[info_elem_key]['scene_num'] = None
    info_json[info_elem_key]['synopsis_num'] = None

In [ ]:
DEBUG = False

for tt_id in (tt_list if not DEBUG else ['tt0047396']):

    subtitle_path = Path(root_path, 'all/subtitle', f'{tt_id}.json')
    with open(subtitle_path, 'r') as f_subtitle:
        subtitle_json = json.load(f_subtitle)

    shot_path = Path(root_path, 'all/shot', f'{tt_id}.json')
    with open(shot_path, 'r') as f_shot:
        shot_json = json.load(f_shot)

    scene_path = Path(root_path, 'all/scene', f'{tt_id}.json')
    with open(scene_path, 'r') as f_scene:
        scene_json = json.load(f_scene)

    info_json[tt_id]['subtitle_num'] = int(subtitle_json[-1]['subtitle_id']) + 1
    info_json[tt_id]['shot_num'] = int(shot_json[-1]['shot_id']) + 1
    info_json[tt_id]['scene_num'] = int(scene_json[-1]['scene_id']) + 1

    if DEBUG :
        print(info_json)
    else:
        with open(info_path, 'w') as f_info:
            json.dump(info_json, f_info, indent=2)


6. aligned 

In [ ]:
DEBUG = False

for tt_id in (tt_list if not DEBUG else ['tt0047396']):

    subtitle_path = Path(root_path, 'all/subtitle', f'{tt_id}.json')
    with open(subtitle_path, 'r') as f_subtitle:
        subtitle_json = json.load(f_subtitle)

    shot_path = Path(root_path, 'all/shot', f'{tt_id}.json')
    with open(shot_path, 'r') as f_shot:
        shot_json = json.load(f_shot)

    scene_path = Path(root_path, 'all/scene', f'{tt_id}.json')
    with open(scene_path, 'r') as f_scene:
        scene_json = json.load(f_scene)

    shot_shot_sum = shot_json[-1]['shot_id']
    scene_shot_sum = scene_json[-1]['shot_end_id'] + 1
    
    if (shot_shot_sum != scene_shot_sum):
        tmp_scene_ele = {
            "scene_id": scene_json[-1]["scene_id"] + 1 ,
            "shot_start_id": scene_json[-1]["shot_end_id"] ,
            "shot_end_id": shot_shot_sum +1
        }
        scene_json.append(tmp_scene_ele)
        if DEBUG:
            print(scene_json)
        else:
            with open(scene_path, 'w') as f_scene:
                json.dump(scene_json, f_scene, indent=2)



7. scene [start, end) -> [start, end]

In [ ]:
DEBUG = False

for tt_id in (tt_list if not DEBUG else ['tt0047396']):

    scene_path = Path(root_path, 'all/scene', f'{tt_id}.json')
    with open(scene_path, 'r') as f_scene:
        scene_json = json.load(f_scene)

    for scene_ele in scene_json:
        shot_id_list = [scene_ele['shot_start_id'], scene_ele['shot_end_id']-1]
        scene_ele['shot_id'] = shot_id_list
        scene_ele.pop('shot_start_id')
        scene_ele.pop('shot_end_id')
    
    if DEBUG:
        print(scene_json)
    else:
        with open(scene_path, 'w') as f_scene:
            json.dump(scene_json, f_scene, indent=2)


8. add /all/synopsis

In [ ]:
story_order = re.compile('(?<=_)\d+')

def story_id_renew(story_ori_id:str) -> int:
    story_id = int(story_order.search(story_ori_id).group(0))
    return story_id

In [ ]:
DEBUG = False

for tt_id in (tt_list if not DEBUG else ['tt0047396']):

    anno_path = Path(ori_path, 'annotation.v1/annotation', f'{tt_id}.json')
    with open(anno_path, 'r') as f_anno:
        anno_json = json.load(f_anno)

    story = anno_json['story']
    syno_json = []

    for story_ele in story:
        syno_json_ele = {
            'synopsis_id': story_id_renew(story_ele['id']) ,
            'shot_id' : [story_ele['shot'][0], story_ele['shot'][1] - 1] ,
            'scentence' : story_ele['description']
            }

        syno_json.append(syno_json_ele)

    if DEBUG:
        print(syno_json)
    else:
        syno_path = Path(root_path, 'all/synopsis', f'{tt_id}.json')
        with open(syno_path, 'w') as f_syno:
            json.dump(syno_json, f_syno, indent=2)

In [ ]:
scene_path, shot_path

In [ ]:
with open(scene_path , 'w') as f_scene:
    json.dump(scene_json, f_scene, indent=2)

9. add shot tail scene_id 

In [ ]:
DEBUG = False

for tt_id in (tt_list if not DEBUG else ['tt0120689']):

    shot_path = Path(root_path, 'all/shot', f'{tt_id}.json')
    with open(shot_path, 'r') as f_shot:
        shot_json = json.load(f_shot)

    last_scene_id = 0
    for shot_ele in shot_json:
        if shot_ele["scene_id"]:
            last_scene_id = shot_ele["scene_id"]
        else: 
            shot_ele["scene_id"] = last_scene_id + 1
    
    if DEBUG:
        print(shot_json)
    else:
        with open(shot_path, 'w') as f_shot:
            json.dump(shot_json, f_shot, indent=2)

10. add info synopsis_num

In [ ]:
info_path = Path(root_path, 'movie_info.json')
with open(info_path, 'r') as f_info:
    info_json = json.load(f_info)

DEBUG = False

for tt_id in (tt_list if not DEBUG else ['tt0047396']):

    synopsis_path = Path(root_path, 'all/synopsis', f'{tt_id}.json')
    with open(synopsis_path, 'r') as f_synopsis:
        synopsis_json = json.load(f_synopsis)

    info_json[tt_id]['synopsis_num'] = int(synopsis_json[-1]['synopsis_id']) + 1

    if DEBUG :
        print(info_json)
    else:
        with open(info_path, 'w') as f_info:
            json.dump(info_json, f_info, indent=2)


11. retrieval cast and cast_annotation

In [ ]:
DEBUG = False

### retrival cast
meta_root_path = Path('/home/jianghui/dataset/MovieNet/meta.v1/meta/')
anno_root_path = Path('/home/jianghui/dataset/MovieNet/annotation.v1/annotation/')
cast_root_path = Path(root_path, 'all/cast/')

prog = re.compile('(?<=_)\d+')

for tt_id in (tt_list if not DEBUG else ['tt0047396']):
    print(f'processing {tt_id} ...')

    meta_path = Path(meta_root_path, f'{tt_id}.json')
    anno_path = Path(anno_root_path, f'{tt_id}.json')
    cast_path = Path(cast_root_path, f'{tt_id}.json')
    
    cast_json = {"cast_pid": [], "cast_bbox":[]}

    with open(meta_path, 'r') as f_meta:
        meta_json = json.load(f_meta)
    
    with open(anno_path, 'r') as f_anno:
        anno_json = json.load(f_anno)

    cast_json['cast_pid'] = [ i['id'] for i in meta_json['cast'] ]
    cast_json['cast_bbox'] = [{
        'cast_id':int(prog.search(i['id']).group(0)),
        'cast_pid':i['pid'],
        'shot_id':i['shot_idx'],
        'img_id':i['img_idx'],
        'bbox':i["body"]["bbox"],
        } for i in anno_json['cast']]

    with open(cast_path, 'w') as f_cast:
        json.dump(cast_json, f_cast, indent=2)



In [ ]:
a = [{"id":123, "key":452346}, {"id":143, "key":4523426}, {"id":323, "key":424356}, {"id":143, "key":4456}]
f = [ {'id' : i['id']} for i in a ]
f

In [ ]:
DEBUG = False
anno_root_path = Path('/home/jianghui/dataset/MovieNet/annotation.v1/annotation/')
no_cast_list = []

for tt_id in (tt_list if not DEBUG else ['tt0047396']):
    anno_path = Path(anno_root_path, f'{tt_id}.json')
    with open(anno_path, 'r') as f_anno:
        anno_json = json.load(f_anno)

    type_dict = set()
    try:
        for i in anno_json['cast']:
            type_dict.add(f'{i["resolution"][0]}-{i["resolution"][1]}')
            info_json[tt_id]['resolution'] = i["resolution"]

        if len(type_dict) != 1: print(f'{tt_id} has more than one resolution')
    except TypeError:
        print(f'{tt_id} do not have cast')
        no_cast_list.append(tt_id)

with open(info_path, 'w') as f_info:
    json.dump(info_json, f_info, indent=2)
    

12. del no cast tt_id

In [ ]:
tt_list = list(set(tt_list).difference(set(no_cast_list)))

In [ ]:
tt_list.sort()

with open(Path(root_path, 'movie_id_list'), 'w') as f_tt_list:
    for tt_line in tt_list:
        f_tt_list.write(tt_line + '\n')

In [ ]:
for no_cast_id in no_cast_list:
    synopsis_path = Path(root_path, 'all/synopsis', f'{no_cast_id}.json')
    synopsis_path.unlink()



13. add cast id to subtitle id

In [ ]:
DEBUG = False

for tt_id in (tt_list if not DEBUG else ['tt0047396']):

    names = locals()
    types = 'cast'
    names[f'{types}_path'] = Path(names[f'{types}_root_path'], f'{tt_id}.json')
    with open(names[f'{types}_path'], 'r') as names[f'f_{types}']:
        names[f'{types}_json'] = json.load(names[f'f_{types}'])
    
    cast_pid_list = cast_json['cast_pid']
    cast_pid_real_list = set()

    for cast_bbox in cast_json['cast_bbox']:
        if cast_bbox['cast_pid'] == None:
            cast_bbox['cast_pid'] = 'others'
        cast_pid_real_list.add(cast_bbox['cast_pid'])
    
    cast_pid_real_list = list(cast_pid_real_list)
    cast_pid_real_list.sort()
    
    cast_json['cast_pid'] = cast_pid_real_list

    names[f'{types}_path'] = Path(names[f'{types}_root_path'], f'{tt_id}.json')
    with open(names[f'{types}_path'], 'w') as names[f'f_{types}']:
        json.dump(names[f'{types}_json'], names[f'f_{types}'], indent=2)

Here make open_json and svae_json to make coding easier

In [1]:
def open_json(types:str, tt_id):
    names[f'{types}_path'] = Path(names[f'{types}_root_path'], f'{tt_id}.json')
    with open(names[f'{types}_path'], 'r') as names[f'f_{types}']:
        names[f'{types}_json'] = json.load(names[f'f_{types}'])
    return names[f'{types}_json']

In [8]:
shot_root_path = Path(root_path, 'all/shot/')
scene_root_path = Path(root_path, 'all/scene/')
subtitle_root_path = Path(root_path, 'all/subtitle/')
synopsis_root_path = Path(root_path, 'all/synopsis/')
cast_root_path = Path(root_path, 'all/cast/')
names = locals()

In [6]:
def save_json(types:str, tt_id):
    names[f'{types}_path'] = Path(names[f'{types}_root_path'], f'{tt_id}.json')
    with open(names[f'{types}_path'], 'w') as names[f'f_{types}']:
        json.dump(names[f'{types}_json'], names[f'f_{types}'], indent=2)


In [ ]:
DEBUG = False

for tt_id in (tt_list if not DEBUG else ['tt0047396']):
    print(f'processing {tt_id} ...')

    cast_json = open_json('cast', tt_id)
    shot_json = open_json('shot', tt_id)
    subtitle_json = open_json('subtitle', tt_id)

    for shot_ele in shot_json:
        shot_ele['cast_pid'] = []

    for cast_bbox_ele in cast_json['cast_bbox']:
        shot_json[cast_bbox_ele['shot_id']]['cast_pid'].append(cast_bbox_ele['cast_pid'])

    for shot_ele in shot_json:
        shot_ele['cast_pid'] = list(set(shot_ele['cast_pid']))

    save_json('shot', tt_id)

    # ---

    for subtitle_ele in subtitle_json:
        subtitle_ele['cast_pid'] = []
    
    for shot_ele in shot_json:
        if shot_ele['subtitle_id'] == []: continue
        try:
            shot_ele['cast_pid']
        except:
            print(f'{shot_ele} do not have cast_pid')
        for subtitle_id in range(shot_ele['subtitle_id'][0], shot_ele['subtitle_id'][1] + 1):
            for cast_pid in shot_ele['cast_pid']:
                subtitle_json[subtitle_id]['cast_pid'].append(cast_pid)

    for subtitle_ele in subtitle_json:
        subtitle_ele['cast_pid'] = list(set(subtitle_ele['cast_pid']))

    save_json('subtitle', tt_id)    


In [ ]:
DEBUG = False

for tt_id in (tt_list if not DEBUG else ['tt0047396']):
    print(f'processing {tt_id} ...')

    subtitle_json = open_json('subtitle', tt_id)

    for subtitle_ele in subtitle_json:
        subtitle_ele.pop('cast_pid')

    save_json('subtitle', tt_id)

In [ ]:
DEBUG = False

for tt_id in (tt_list if not DEBUG else ['tt0047396']):
    print(f'processing {tt_id} ...')

    shot_json = open_json('shot', tt_id)
    synopsis_json = open_json('synopsis', tt_id)

    for synopsis_ele in synopsis_json:
        scene_start_id = shot_json[synopsis_ele['shot_id'][0]]['scene_id']
        scene_end_id = shot_json[synopsis_ele['shot_id'][1]]['scene_id']
        synopsis_ele['scene_id']=[scene_start_id, scene_end_id]
        
    save_json('synopsis', tt_id)

In [11]:
DEBUG = False

for tt_id in (tt_list if not DEBUG else ['tt0047396']):
    print(f'processing {tt_id} ...')

    cast_json = open_json('cast', tt_id)

    cast_json['cast_pid_list'] = cast_json.pop('cast_pid')
        
    save_json('cast', tt_id)

processing tt0047396 ...
processing tt0048545 ...
processing tt0049730 ...
processing tt0052357 ...
processing tt0056923 ...
processing tt0061418 ...
processing tt0061722 ...
processing tt0062622 ...
processing tt0063442 ...
processing tt0065214 ...
processing tt0065724 ...
processing tt0066026 ...
processing tt0067116 ...
processing tt0068646 ...
processing tt0070735 ...
processing tt0071315 ...
processing tt0071562 ...
processing tt0072684 ...
processing tt0073195 ...
processing tt0074285 ...
processing tt0075148 ...
processing tt0076759 ...
processing tt0077405 ...
processing tt0078788 ...
processing tt0078841 ...
processing tt0079417 ...
processing tt0080684 ...
processing tt0081505 ...
processing tt0082089 ...
processing tt0082971 ...
processing tt0083658 ...
processing tt0083929 ...
processing tt0084726 ...
processing tt0086190 ...
processing tt0086250 ...
processing tt0086879 ...
processing tt0087332 ...
processing tt0088847 ...
processing tt0088944 ...
processing tt0090022 ...
